In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import transforms
import torchvision.datasets as datasets

# Set up environment variables
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12355'

# Initialize process group
num_gpus = torch.cuda.device_count()
dist.init_process_group(backend='nccl', world_size=num_gpus, rank=torch.cuda.current_device())

# Define paths
data_path = '/path/to/your/dataset'-------------------------------------------------->Pothole Annotated dataset
train_annotations = os.path.join(data_path, 'train_annotations.json')
val_annotations = os.path.join(data_path, 'val_annotations.json')

# Hyperparameters
batch_size = 8
num_epochs = 50
learning_rate = 0.001

# Data preprocessing
transform = transforms.Compose([
    transforms.Resize((224,224)),
transforms.RandomHorizontalFlip(),
transforms.ToTensor(),
transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])


train_data = datasets.CocoDetection(root=os.path.join(data_path, 'train'), annFile=train_annotations, transform=transform)
val_data = datasets.CocoDetection(root=os.path.join(data_path, 'val'), annFile=val_annotations, transform=transform)

train_sampler = DistributedSampler(train_data, num_replicas=num_gpus, rank=torch.cuda.current_device())
val_sampler = DistributedSampler(val_data, num_replicas=num_gpus, rank=torch.cuda.current_device())

train_loader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=4)
val_loader = DataLoader(val_data, batch_size=batch_size, sampler=val_sampler, num_workers=4)

device = torch.device(f'cuda:{torch.cuda.current_device()}')
model = fasterrcnn_resnet50_fpn(pretrained=False, num_classes=2).to(device)
model = nn.parallel.DistributedDataParallel(model, device_ids=[torch.cuda.current_device()])
oss function and optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=learning_rate, momentum=0.9, weight_decay=0.0005)


for epoch in range(num_epochs):
model.train()
train_sampler.set_epoch(epoch)
running_loss = 0.0
for i, (inputs, targets) in enumerate(train_loader):
    inputs = list(img.to(device) for img in inputs)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    optimizer.zero_grad()

    losses = model(inputs, targets)
    loss = sum(l for l in losses.values())
    loss.backward()
    optimizer.step()

  running_loss += loss.item()
print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Validate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in val_loader:
        inputs = list(img.to(device) for img in inputs)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
 outputs = model(inputs)
      for idx, output in enumerate(outputs):
            gt_boxes = targets[idx]['boxes']
            pred_boxes = output['boxes']

      for gt_box in gt_boxes:
           for pred_box in pred_boxes:
                iou = get_iou(gt_box, pred_box)
              if iou >= 0.5:
                    correct += 1
                      break
        total += sum([len(t['boxes']) for t in targets])

print(f"Validation accuracy: {100 * correct / total:.2f}%")

torch.save(model.module.state_dict(), "pothole_detector.pth")

def get_iou(boxA, boxB):
# Determine the (x, y)-coordinates of the intersection rectangle
xA = max(boxA[0], boxB[0])
yA = max(boxA[1], boxB[1])
xB = min(boxA[2], boxB[2])
yB = min(boxA[3], boxB[3])
max(0, yB - yA + 1)
boxA_area = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
boxB_area = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
iou = inter_area / float(boxA_area + boxB_area - inter_area)

return iou
